# 04_tfl_disruptions_backfill_fast.ipynb

Latency-focused TfL disruptions backfill:
- Uses `point` (string like `"[lon,lat]"`) instead of `geometry`
- Vectorized distance matching (no nested loops)
- Aggregates to hourly features per `(point_id, tfl_time_utc)`


In [1]:
import os
import ast
import numpy as np
import pandas as pd
import requests
import hopsworks

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 200)


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# ================= CONFIG =================
METADATA_FG_NAME = os.getenv("METADATA_FG_NAME", "traffic_points_metadata")
METADATA_FG_VERSION = int(os.getenv("METADATA_FG_VERSION", "1"))

TRAFFIC_FG_NAME = os.getenv("TRAFFIC_FG_NAME", "traffic_flow_fg")
TRAFFIC_FG_VERSION = int(os.getenv("TRAFFIC_FG_VERSION", "1"))

TFL_FG_NAME = os.getenv("TFL_FG_NAME", "tfl_disruptions_hourly_fg")
TFL_FG_VERSION = int(os.getenv("TFL_FG_VERSION", "1"))

TFL_BASE_URL = "https://api.tfl.gov.uk"
TFL_APP_ID = os.getenv("TFL_APP_ID", "")
TFL_APP_KEY = os.getenv("TFL_APP_KEY", "")

TFL_START_DATE = os.getenv("TFL_START_DATE", "2026-01-04")  # YYYY-MM-DD
TFL_END_DATE = os.getenv("TFL_END_DATE", "2026-01-09")      # YYYY-MM-DD

RADIUS_KM = float(os.getenv("TFL_RADIUS_KM", "0.5"))
MAX_BACKFILL_DAYS = int(os.getenv("TFL_MAX_BACKFILL_DAYS", "14"))


In [12]:
# ============== CONNECT TO HOPSWORKS ==============
project = hopsworks.login(
    host="eu-west.cloud.hopsworks.ai",
    project="London_traffic"
)
fs = project.get_feature_store()


2026-01-09 13:58:56,816 INFO: Closing external client and cleaning up certificates.
2026-01-09 13:58:56,834 INFO: Connection closed.
2026-01-09 13:58:56,839 INFO: Initializing external client
2026-01-09 13:58:56,839 INFO: Base URL: https://eu-west.cloud.hopsworks.ai:443
2026-01-09 13:58:58,082 INFO: Python Engine initialized.

Logged in to project, explore it here https://eu-west.cloud.hopsworks.ai:443/p/3209


In [13]:
# ============== READ MONITORING POINTS ==============
meta_fg = fs.get_feature_group(name=METADATA_FG_NAME, version=METADATA_FG_VERSION)
points_df = meta_fg.read()

required = ["point_id", "latitude", "longitude"]
missing = [c for c in required if c not in points_df.columns]
if missing:
    raise ValueError(f"Missing metadata columns: {missing}")

points_df = points_df[required].copy()
points_df["point_id"] = points_df["point_id"].astype(str)
points_df["latitude"] = pd.to_numeric(points_df["latitude"], errors="coerce")
points_df["longitude"] = pd.to_numeric(points_df["longitude"], errors="coerce")
points_df = points_df.dropna(subset=["latitude", "longitude"]).drop_duplicates("point_id")

print("Monitoring points:", len(points_df))


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.00s) 
Monitoring points: 200


In [14]:
# ============== DATE RANGE ==============
def infer_date_range_from_traffic(max_days: int) -> (str, str):
    traffic_fg = fs.get_feature_group(name=TRAFFIC_FG_NAME, version=TRAFFIC_FG_VERSION)
    tdf = traffic_fg.read()
    if "ts_10m" not in tdf.columns:
        raise ValueError("Traffic FG must contain 'ts_10m' to infer date range.")
    tdf["ts_10m"] = pd.to_datetime(tdf["ts_10m"], utc=True, errors="coerce")
    tdf = tdf.dropna(subset=["ts_10m"])

    end = tdf["ts_10m"].max().normalize()
    start = tdf["ts_10m"].min().normalize()
    if (end - start).days > max_days:
        start = end - pd.Timedelta(days=max_days)

    return start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d")

if TFL_START_DATE and TFL_END_DATE:
    start_date, end_date = TFL_START_DATE, TFL_END_DATE
else:
    start_date, end_date = infer_date_range_from_traffic(MAX_BACKFILL_DAYS)

print("TfL backfill range:", start_date, "->", end_date)


TfL backfill range: 2026-01-04 -> 2026-01-09


In [15]:
# ============== FETCH TFL DISRUPTIONS ==============
def fetch_tfl_disruptions(start_date: str, end_date: str) -> list:
    url = f"{TFL_BASE_URL}/Road/all/Disruption"
    params = {"startDate": start_date, "endDate": end_date}
    if TFL_APP_ID:
        params["app_id"] = TFL_APP_ID
    if TFL_APP_KEY:
        params["app_key"] = TFL_APP_KEY

    r = requests.get(url, params=params, timeout=60)
    r.raise_for_status()
    return r.json()

raw = fetch_tfl_disruptions(start_date, end_date)
raw_df = pd.DataFrame(raw)

print("Raw disruptions:", len(raw_df))
raw_df.head()


Raw disruptions: 102


,$type,id,url,point,severity,ordinal,category,subCategory,comments,currentUpdate,currentUpdateDateTime,corridorIds,startDateTime,endDateTime,lastModifiedTime,levelOfInterest,location,status,geography,geometry,streets,isProvisional,hasClosures,roadDisruptionLines,roadDisruptionImpactAreas,recurringSchedules
0,"Tfl.Api.Presentation.Entities.RoadDisruption, ...",TIMS-206772,/Road/All/Disruption/TIMS-206772,"[0.212921,51.592491]",Moderate,1,Works,TfL works,Gallows Corner Flyover Refurbishment - [A12] E...,Use alternative routes. There are no reported ...,2025-12-19T23:44:05Z,[a12],2025-03-15T21:00:00Z,2026-03-31T21:59:00Z,2025-12-19T23:45:32Z,High,"[A12] EASTERN AVENUE EAST (RM2 ,RM3 ) (Havering)",Active,"{'type': 'Point', 'coordinates': [0.212921, 51...","{'type': 'MultiPolygon', 'coordinates': [[[[0....",[{'$type': 'Tfl.Api.Presentation.Entities.Stre...,False,False,[],[],[]
1,"Tfl.Api.Presentation.Entities.RoadDisruption, ...",TIMS-213893,/Road/All/Disruption/TIMS-213893,"[-0.125602,51.440164]",Moderate,2,Works,TfL works,[A23] Streatham Hill (Both directions) between...,Traffic is flowing well.,2025-12-19T20:27:41Z,[a23],2025-08-03T21:00:00Z,2026-07-06T05:00:00Z,2025-12-19T20:27:55Z,High,"[A23] STREATHAM HILL (SW16 ,SW2 ) (Lambeth)",Active,"{'type': 'Point', 'coordinates': [-0.125602, 5...","{'type': 'Polygon', 'coordinates': [[[-0.12807...",[{'$type': 'Tfl.Api.Presentation.Entities.Stre...,False,False,[],[],[]
2,"Tfl.Api.Presentation.Entities.RoadDisruption, ...",TIMS-200007,/Road/All/Disruption/TIMS-200007,"[-0.087718,51.512021]",Moderate,3,Works,Borough works,King William Street (Southbound) at the juncti...,Use an alternative route. Delays are possible ...,2025-12-19T07:19:40Z,[bishopsgate cross route],2024-07-08T07:00:00Z,2026-04-30T19:00:00Z,2025-12-19T07:19:48Z,High,"KING WILLIAM STREET (EC3V,EC4N,EC4R) (City of ...",Active,"{'type': 'Point', 'coordinates': [-0.087718, 5...","{'type': 'Polygon', 'coordinates': [[[-0.08917...",[{'$type': 'Tfl.Api.Presentation.Entities.Stre...,False,False,[],[],[]
3,"Tfl.Api.Presentation.Entities.RoadDisruption, ...",TIMS-216408,/Road/All/Disruption/TIMS-216408,"[-0.13886,51.530139]",Moderate,4,Works,Network Rail works,[A400] Hampstead Road (Both directions) at the...,Delays are possible.,2025-12-19T16:04:33Z,[inner ring],2025-09-04T13:21:00Z,2026-02-28T23:59:00Z,2025-12-19T16:04:33Z,High,[A400] HAMPSTEAD ROAD (NW1 ) (Camden),Active,"{'type': 'Point', 'coordinates': [-0.13886, 51...",NaN,NaN,False,False,[],[],[]
4,"Tfl.Api.Presentation.Entities.RoadDisruption, ...",TIMS-204461,/Road/All/Disruption/TIMS-204461,"[-0.122625,51.494512]",Moderate,5,Works,TfL works,[A3203] Lambeth Bridge (Eastbound) between [A3...,Delays are possible.,2025-12-19T07:23:28Z,[southern river route],2025-12-05T20:45:00Z,2026-06-15T14:00:00Z,2025-12-19T07:23:37Z,High,"[A3203] LAMBETH BRIDGE (SE1 ,SW1P ) (Lambeth,W...",Active,"{'type': 'Point', 'coordinates': [-0.122625, 5...","{'type': 'Polygon', 'coordinates': [[[-0.12573...",[{'$type': 'Tfl.Api.Presentation.Entities.Stre...,False,False,[],[],[]


In [16]:
# ============== PARSE LOCATION (POINT STRING) ==============
def parse_point_string(s):
    # Expected format: "[lon,lat]"
    if isinstance(s, str):
        try:
            v = ast.literal_eval(s)
            if isinstance(v, (list, tuple)) and len(v) == 2:
                lon, lat = float(v[0]), float(v[1])
                return lat, lon
        except Exception:
            return np.nan, np.nan
    return np.nan, np.nan

needed_cols = ["id", "category", "subCategory", "severity", "ordinal", "status", "startDateTime", "endDateTime", "point"]
for c in needed_cols:
    if c not in raw_df.columns:
        raw_df[c] = np.nan

raw_df["startDateTime"] = pd.to_datetime(raw_df["startDateTime"], utc=True, errors="coerce")
raw_df["endDateTime"] = pd.to_datetime(raw_df["endDateTime"], utc=True, errors="coerce")
raw_df = raw_df.dropna(subset=["startDateTime"]).copy()

raw_df[["lat", "lon"]] = raw_df["point"].apply(lambda s: pd.Series(parse_point_string(s)))
raw_df = raw_df.dropna(subset=["lat", "lon"]).copy()

print("Disruptions with lat/lon:", len(raw_df))
raw_df[["id", "lat", "lon", "category", "ordinal", "status"]].head()


Disruptions with lat/lon: 102


,id,lat,lon,category,ordinal,status
0,TIMS-206772,51.592491,0.212921,Works,1,Active
1,TIMS-213893,51.440164,-0.125602,Works,2,Active
2,TIMS-200007,51.512021,-0.087718,Works,3,Active
3,TIMS-216408,51.530139,-0.138860,Works,4,Active
4,TIMS-204461,51.494512,-0.122625,Works,5,Active


In [17]:
# ============== VECTORIZED SPATIAL MATCHING ==============
def haversine_km_matrix(lat1, lon1, lat2, lon2):
    R = 6371.0
    lat1 = np.radians(lat1)[:, None]
    lon1 = np.radians(lon1)[:, None]
    lat2 = np.radians(lat2)[None, :]
    lon2 = np.radians(lon2)[None, :]

    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2
    c = 2.0 * np.arcsin(np.sqrt(a))
    return R * c

d_lat = raw_df["lat"].to_numpy(dtype=float)
d_lon = raw_df["lon"].to_numpy(dtype=float)

p_lat = points_df["latitude"].to_numpy(dtype=float)
p_lon = points_df["longitude"].to_numpy(dtype=float)
p_ids = points_df["point_id"].to_numpy(dtype=str)

dist = haversine_km_matrix(d_lat, d_lon, p_lat, p_lon)
d_idx, p_idx = np.where(dist <= RADIUS_KM)

links = pd.DataFrame({
    "point_id": p_ids[p_idx],
    "category": raw_df.iloc[d_idx]["category"].to_numpy(),
    "ordinal": pd.to_numeric(raw_df.iloc[d_idx]["ordinal"], errors="coerce").fillna(0).astype(int).to_numpy(),
    "status": raw_df.iloc[d_idx]["status"].to_numpy(),
    "start": raw_df.iloc[d_idx]["startDateTime"].to_numpy(),
    "end": raw_df.iloc[d_idx]["endDateTime"].to_numpy(),
})

print("Point-disruption links:", len(links))
links.head()


Point-disruption links: 44


,point_id,category,ordinal,status,start,end
0,8618,Works,3,Active,2024-07-08 07:00:00+00:00,2026-04-30 19:00:00+00:00
1,38269,Works,3,Active,2024-07-08 07:00:00+00:00,2026-04-30 19:00:00+00:00
2,38368,Works,4,Active,2025-09-04 13:21:00+00:00,2026-02-28 23:59:00+00:00
3,17567,Works,5,Active,2025-12-05 20:45:00+00:00,2026-06-15 14:00:00+00:00
4,26146,Works,6,Recurring Works,2026-01-05 22:00:00+00:00,2026-01-06 05:00:00+00:00


In [18]:
# ============== HOURLY EXPANSION + AGGREGATION ==============
if len(links) == 0:
    agg_df = pd.DataFrame(columns=[
        "point_id", "tfl_time_utc",
        "disruption_count", "is_works", "is_incident", "is_active",
        "max_ordinal"
    ])
else:
    links = links.copy()
    links["end_filled"] = links["end"]
    links.loc[pd.isna(links["end_filled"]), "end_filled"] = links["start"] + np.timedelta64(1, "h")

    links["start_hour"] = pd.to_datetime(links["start"], utc=True).dt.floor("H")
    links["end_hour"] = pd.to_datetime(links["end_filled"], utc=True).dt.floor("H")

    links["hours"] = links.apply(lambda r: pd.date_range(r["start_hour"], r["end_hour"], freq="H"), axis=1)
    hourly = links.explode("hours").rename(columns={"hours": "tfl_time_utc"}).copy()

    hourly["disruption_count"] = 1
    hourly["is_works"] = (hourly["category"] == "Works").astype(int)
    hourly["is_incident"] = (hourly["category"] == "Incident").astype(int)
    hourly["is_active"] = (hourly["status"].astype(str).str.lower() == "active").astype(int)
    hourly["max_ordinal"] = pd.to_numeric(hourly["ordinal"], errors="coerce").fillna(0).astype(int)

    agg_df = (
        hourly.groupby(["point_id", "tfl_time_utc"], as_index=False)
        .agg(
            disruption_count=("disruption_count", "sum"),
            is_works=("is_works", "max"),
            is_incident=("is_incident", "max"),
            is_active=("is_active", "max"),
            max_ordinal=("max_ordinal", "max"),
        )
    )

print("Hourly TfL features shape:", agg_df.shape)
agg_df.head()


Hourly TfL features shape: (61709, 7)


,point_id,tfl_time_utc,disruption_count,is_works,is_incident,is_active,max_ordinal
0,16616,2026-01-07 07:00:00+00:00,1,1,0,0,89
1,16616,2026-01-07 08:00:00+00:00,1,1,0,0,89
2,16616,2026-01-07 09:00:00+00:00,1,1,0,0,89
3,16616,2026-01-07 10:00:00+00:00,1,1,0,0,89
4,16616,2026-01-07 11:00:00+00:00,1,1,0,0,89


In [22]:
# ============== WRITE FEATURE GROUP ==============
tfl_fg = fs.get_or_create_feature_group(
    name=TFL_FG_NAME,
    version=TFL_FG_VERSION,
    primary_key=["point_id", "tfl_time_utc"],
    event_time="tfl_time_utc",
    description="Hourly TfL road disruption features (backfilled, low-latency point-based matching)."
)

tfl_fg.insert(
    agg_df
)

print(f" TfL disruptions backfill written to {TFL_FG_NAME} v{TFL_FG_VERSION}")


Uploading Dataframe: 100.00% |██████████| Rows 61709/61709 | Elapsed Time: 00:02 | Remaining Time: 00:00


 TfL disruptions backfill written to tfl_disruptions_hourly_fg v1


Use fg.materialization_job.run(args=-op offline_fg_materialization -path hdfs:///Projects/London_traffic/Resources/jobs/tfl_disruptions_hourly_fg_1_offline_fg_materialization/config_1767963575279) to trigger the materialization job again.


In [24]:
agg_df.head()

,point_id,tfl_time_utc,disruption_count,is_works,is_incident,is_active,max_ordinal
0,16616,2026-01-07 07:00:00+00:00,1,1,0,0,89
1,16616,2026-01-07 08:00:00+00:00,1,1,0,0,89
2,16616,2026-01-07 09:00:00+00:00,1,1,0,0,89
3,16616,2026-01-07 10:00:00+00:00,1,1,0,0,89
4,16616,2026-01-07 11:00:00+00:00,1,1,0,0,89
